In [89]:
# Importa a biblioteca para criação de classes de dados
from dataclasses import dataclass

# Importação dos tipos de dados para o type hint
from typing import Callable

# Importação da biblioteca de números aleatórios
from random import randint
from random import sample
from random import random

# Importação da biblioteca de impressão bonita
from pprint import pprint

# Importação da biblioteca de cópia
from copy import deepcopy

In [90]:
@dataclass
class Cromossomo:
    dados: list[int]
    fitness: float = 0

In [91]:
@dataclass
class Config:
    tam_cromossomo: int
    tam_populacao: int

    max_iteracoes: int
    max_congelamento : int

    fitness: Callable[[Cromossomo], float]

    selecionar_pais: Callable[[list[Cromossomo]], list[tuple[Cromossomo, Cromossomo]]]

    aplicar_cruzamento: Callable[[list[tuple[Cromossomo, Cromossomo]]], list[Cromossomo]]
    aplicar_mutacao: Callable[[list[Cromossomo], float], None]
    taxa_mutacao: float

    selecionar_sobreviventes: Callable[[list[Cromossomo]], list[Cromossomo]]

In [92]:
def inicializar_populacao(tam_populacao, tam_cromossomo) -> list[Cromossomo]:
    P0 = []
    for i in range(tam_populacao):
        cromossomo = Cromossomo([randint(0, 1) for j in range(tam_cromossomo)])
        P0.append(cromossomo)
    return P0

In [93]:
# Seleção por torneio - com reposição
# - Seleciona k pais e o tiver melhor fitness é selecionado para formar casal
def torneio(populacao: list[Cromossomo], tam_torneio: int = 3) -> list[tuple[Cromossomo, Cromossomo]]:
    casais = []

    for i in range(len(populacao) // 2):

        torneio1 = sample(populacao, tam_torneio * 2)
        pai1 = sorted(torneio1, key=lambda x: x.fitness, reverse=True)[0]

        torneio2 = sample(populacao, tam_torneio)
        pai2 = sorted(torneio2, key=lambda x: x.fitness, reverse=True)[0]

        # TODO: evitar que o selecionado no pai1 seja selecionado como pai2

        casais.append((pai1, pai2))

    return casais

In [94]:
def crossover_1_corte(casais):
    filhos = []

    for par in casais:

        # Desempacota os pais
        pai1 = par[0].dados
        pai2 = par[1].dados

        # Sorteia o ponto de corte
        corte = randint(1, len(pai1) - 1)

        filho1 = pai1[:corte] + pai2[corte:]
        filho2 = pai2[:corte] + pai1[corte:]

        filhos.append(Cromossomo(filho1))
        filhos.append(Cromossomo(filho2))

    return filhos

In [95]:
def mutacao(cromossomos: list[Cromossomo], taxa_mutacao: float):
    for cromossomo in cromossomos:
        for i, c in enumerate(cromossomo.dados):
            if random() < taxa_mutacao:
                if c == 1:
                    cromossomo.dados[i] = 0
                else:
                    cromossomo.dados[i] = 1

In [96]:
def elitismo(populacao):
    populacao.sort(key=lambda x: x.fitness, reverse=True)
    return populacao[:len(populacao)//2]

In [97]:
def algoritmo_genetico(config: Config) -> list[int]:
    """
    Implementação do algoritmo genético clássico.

    Arguments:
        config: Parâmetros de configuração do AG.
    """

    # 1. t = 0
    t = 0

    # 2. Inicializar a população inicial P_0
    P = inicializar_populacao(config.tam_populacao, config.tam_cromossomo)

    # a. Avaliar a população(Pt)
    for c in P:
        c.fitness = config.fitness(c)

    # Salva o melhor indivíduo
    P.sort(key=lambda x: x.fitness, reverse=True)

    melhor_individuo = deepcopy(P[0])
    cont_congelamento = 0


    # 3. Enquanto critério de parada == falso
    terminou = False
    while (t < config.max_iteracoes
               and cont_congelamento < config.max_congelamento):

        # b. P' = Selecionar pais(Pt)
        casais = config.selecionar_pais(P)

        # c. F = Aplicar recombinação e mutação(P')
        F = config.aplicar_cruzamento(casais)
        config.aplicar_mutacao(F, config.taxa_mutacao)

        # d. Avaliar a população(F)
        for c in F:
            c.fitness = config.fitness(c)

        # e. Pt+1 = Selecionar sobreviventes(Pt + F)
        P = config.selecionar_sobreviventes(P + F)

        # Salva o melhor indivíduo
        P.sort(key=lambda x: x.fitness, reverse=True)

        if P[0].fitness > melhor_individuo.fitness:
            melhor_individuo = deepcopy(P[0])
            cont_congelamento = 0
        else:
            cont_congelamento += 1

        print(f'{t:04d} - {melhor_individuo.fitness} - {melhor_individuo.dados}')

        # f. t = t + 1
        t = t + 1

    return melhor_individuo

In [98]:
# Problema de maximizar a quantidade de 1s no vetor

def maximizar_1s(cromossomo: Cromossomo) -> int:
    """
    Função de avaliação do problema. É nessa função que o cromossomo é interpretado
    para calcular a aptidão e verificar se está atingindo o objetivo.
    """
    return sum(cromossomo.dados)



In [110]:
cartoes = (1,2,3,4,5,6,7,8,9,10)

MAX_PILHA1 = 36
MAX_PILHA2 = 360
VALOR_CHAVE = 396

#0 -> Pilha 1//1 -> Pilha 2 se soma + produto for 396 significa que esta certo
def fitness_cartao(cromossomo: Cromossomo) -> float:
  soma = 0
  produto = 1
  for i in range(10):
    if cromossomo.dados[i] == 1:
      produto *= cartoes[i]
    else:
      soma += cartoes[i]

  if produto > MAX_PILHA2 or soma > MAX_PILHA1 or produto + soma > VALOR_CHAVE:
    return -1

  return produto + soma


In [109]:
# Parâmetros de configuração do AG
config = Config(
    tam_cromossomo=10,
    tam_populacao=1000,

    max_iteracoes=1000,
    max_congelamento=100,

    fitness=fitness_cartao,

    taxa_mutacao=0.05,

    selecionar_pais=torneio,

    aplicar_cruzamento=crossover_1_corte,
    aplicar_mutacao=mutacao,
    selecionar_sobreviventes=elitismo,
)

# Execução do algoritmo genético
solucao = algoritmo_genetico(config)

0000 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0001 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0002 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0003 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0004 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0005 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0006 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0007 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0008 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0009 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0010 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0011 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0012 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0013 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0014 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0015 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0016 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0017 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0018 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0019 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0020 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0021 - 396 - [1, 0, 1, 1, 1, 1, 0, 0, 0, 0]
0022 - 396 - [1, 0, 1, 1, 1, 1, 